
# Les listes de listes 

Si l'on relit la définition d'une liste, *objet qui peut contenir plusieurs objets*, rien ne nous empêche de placer des listes dans des listes. Par exemple :

In [ ]:
maliste = [["Fer",26],["Ag",47],["Ca",20],["Al",13]]

Afin de récupérer une sous-liste, on utilise son index :

In [ ]:
print(maliste[0])
print(maliste[3])

Mais il est possible de récupérer directement l'élement d'une sous-liste, en utilisant un premier index pour sélectionner la sous-liste, puis un second pour sélectionner l'élément de la sous-liste :

In [ ]:
print(maliste[0][1])
print("L'élément",maliste[0][0],"a",maliste[0][1],"protons.")



# Les compréhensions de liste

On appelle **compréhension de liste** la technique suivante qui permet de créer, modifier ou de filtrer une liste très facilement.

## Opérations simples :
Imaginons que nous voulons créer une liste en mettant au carré chaque élément d'une liste d'origine. L'idée qui vient tout de suite à l'esprit est de faire une boucle comme ceci (on remarque au passage l'intérêt de créer une liste vide, ici appelée `carre` : ceci permet d'utiliser la fonction `append()` sur l'objet `carre`, qui existe bien)

In [ ]:
liste = [0,1,3,-1,5, 6] # initialisation
carre = [] #initialisation de la liste qui contiendra les carrés

for elt in liste :
    carre.append(elt**2)
print(carre)

Avec *python*, il est possible de faire cette même boucle en une ligne de commande, que nous commenterons après :

In [ ]:
liste = [0,1,3,-1,5, 6] # initialisation

carre = [elt**2 for elt in liste]
print(carre)

Parcourons la seconde ligne de **droite à gauche** :

- grâce à la commande `for elt in liste`, la variable `elt` parcourt successivement les différents élements contenues dans la liste. 
- pour chaque élément, on calcule `elt**2`.
- les `[]` indiquent que les résultats précédents sont stockés dans une liste
- liste que l'on affecte à la variable `carre` grâce à l'opérateur `=`

## Filtres sur une liste

Il est également possible d'ajouter une condition pour ne sélectionner qu'une partie des élements d'une liste.

In [ ]:
nombres = [-11,10,9,-8,12,-4,20]

nombres_positifs = [elt for elt in nombres if elt > 0]
print(nombres_positifs)

- ici `elt` parcourt les élements de la liste `nombres` en ne considérant que les nombres positifs.
- pour chaque élement retenu, on retourne sa valeur pour former une nouvelle liste `nombres_positifs`.

# Fonctions sur les listes :

## Fonctions usuelles 
Il existe plusieurs fonctions usuelles bien pratiques à connaître. Vous allez les découvrir au fur et à mesure. En voici quelques-unes. Leur nom est assez explicite :

In [ ]:
nombres = [-11,10,9,-8,12,-4,20]

sum(nombres) # fait la somme des élements d'une liste


In [ ]:
max(nombres) # retourne le maximum d'une liste

In [ ]:
min(nombres) # retourne le minimum d'une liste

In [ ]:
sorted(nombres) # ordonne du plus petit au plus grand les éléments d'une liste

In [ ]:
mots = ['bonjour','girafe','schtroumpf','tagada', 'ananas', 'zoo']

sorted(mots) # ordonne dans l'ordre alphabétique les éléments d'une liste de chaînes de caractères.

**Attention**, `sorted(mots)` est une fonction (pas une méthode) et `mots` est son argument. La liste `mots` n'est pas modifiée par le programme précédent :

In [ ]:
print(mots)

## Fonctions provenant de librairies
Il existe beaucoup de librairies contenant des fonctions s'appliquant aux listes. À vous de les chercher en fonction de vos besoins. En voici une :

In [ ]:
from statistics import mean, median
median(nombres) # retourne la valeur médiane
mean(nombres) # retourne la valeur moyenne

---
## Exercice 1 : Tableau périodique

Chaque élément de la liste suivante est une liste à deux éléments, contenant le symbole et le numéro atomique Z d'une espèce atomique.

Atomes = [["Fer",26],["Ag",47],["Ca",20],["Al",13],["Ne",10],["O",8],]

Ajouter à cette liste l'or de symbole "Au" et Z=79 puis afficher la liste complétée.

In [ ]:
Atomes = [["Fer",26],["Ag",47],["Ca",20],["Al",13],["Ne",10],["O",8]]

Pour trier par ordre alphabétique du nom d'élément, on peut utiliser la fonction `sorted`

In [ ]:
sorted(Atomes)

Écrire un programme qui trie les élements chimiques précédents par ordre croissant de numéro atomique.

Algorithme :
- Écrire un programme qui crée une liste dans laquelle numéro et nom sont inversés :

    [[26, 'Fer'], [47, 'Ag'], [20, 'Ca'], [13, 'Al'], [10, 'Ne'], [8, 'O'], [79, 'Au']]

- Trier cette nouvelle liste avec la commande `sorted`


In [ ]:
Atomes = [["Fer",26],["Ag",47],["Ca",20],["Al",13],["Ne",10],["O",8],["Au",79]]

---
## Exercice 2
Chaque élément de la liste suivante est une liste à quatre éléments donnant le prénom, le nom, le prénom, la date de naissance et la date de décès de différentes personnalités du monde de la physique.

    liste=[['James Clerk', 'Maxwell', 1831, 1879], ['Albert', 'Einstein', 1879, 1955], ['Isaac', 'Newton', 1643, 1727]]

Ajouter un élément à la liste pour Lise Meitner (1878-1968) et afficher la liste complète.

In [ ]:
liste=[['James Clerk', 'Maxwell', 1831, 1879], ['Albert', 'Einstein', 1879, 1955], ['Isaac', 'Newton', 1643, 1727]]

Écrire un programme affichant la liste des noms par ordre de date de naissance croissante, sous la forme

    Isaac Newton de 1878 à 1968
    James Clerk Maxwell de 1831 à 1879
    (etc...)